In [20]:
import pandas as pd
import httpx
from selectolax.parser import HTMLParser
from bs4 import BeautifulSoup



In [21]:
info = []
errors=[]

In [22]:
url = 'https://www.agenziaentrate.gov.it/portale/web/guest/siti-web-regionali'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0"}

resp = httpx.get(url, headers=headers)

if resp.status_code == 200:
    html = HTMLParser(resp.text)
    regioni = html.css("li.submenu")

    region_counter = 0
    for reg in regioni:
        region_name = reg.text().strip()
        Link = reg.css_first('a').attributes['href']
        print(region_name)

        resp = httpx.get(Link, headers=headers)
        if resp.status_code == 200:
            if  region_name == 'Alto Adige (Italiano-Tedesco)': 
                html = HTMLParser(resp.text)
                italiano = html.css("a.btn.btn-info.btn-entrypoint")
                link_ita = italiano[0].attributes['href']
                full_link_ita = httpx.URL(url).join(link_ita)
                print(f"Following link: {full_link_ita}")
                resp = httpx.get(full_link_ita, headers=headers)
                
            html = HTMLParser(resp.text)
            province = html.css("a.btn.btn-primary.btn-entrypoint")
            for prov in province:
                print(prov.text())
                link_prov = prov.attributes['href']
                full_link = httpx.URL(url).join(link_prov)
                print(f"Following link: {full_link}")

                resp_prov = httpx.get(full_link, headers=headers)
                if resp_prov.status_code == 200:

                    # da modificare per prendere tutti gli uffici disponibili
                    html_menu = HTMLParser(resp_prov.text)
                    menu = html_menu.css("li.nav-item.subli")
                    
                    for direnzione in menu:
                        link_direzioni = direnzione.css('a')
                        link_dir = link_direzioni[0].attributes['href']
                        print(f"Following link: {link_dir}")
                        resp_dir = httpx.get(link_dir, headers=headers)

                        html_content = resp_dir.text
                        soup = BeautifulSoup(html_content, 'html.parser')

                        uffici = soup.select("h1.underline")
                        ufficio = uffici[0].text

                        if 'Ufficio Controlli' in ufficio or 'Ufficio Legale' in ufficio:
                            print(' ')

                        else:    

                            # Locate the specific div containing "Informazioni utili"
                            divs = soup.select('div.panel.panel-default.panel-collapsible')
                            

                            for div in divs:
                                informazioni_div=''
                                title_element = div.select_one('a.panel-title')
                                if title_element and 'Informazioni utili' in title_element.text:
                                    informazioni_div = div
                                    break

                            try:
                                if informazioni_div:

                                    paragraphs = informazioni_div.find_all('p')

                                for paragraph in paragraphs:
                                    info.append({
                                        'region': region_name,
                                        'province': ufficio,
                                        'info': paragraph.get_text()
                                    })
                                    paragraph=''
                                informazioni_div='' 
                                paragraphs=''   

                            except Exception as e:
                                print(f"Error finding 'Collapse-2' <p> element: {e}")
                                errors.append({
                                        'region': region_name,
                                        'province': prov.text()})
                else:
                    print(f"Failed to fetch province page: {full_link}")
        else:
            print(f"Failed to fetch region page.")
else:
    print("Failed to fetch main page.")

Abruzzo
L'Aquila
Following link: https://www.agenziaentrate.gov.it/portale/uffici/-/uffici/uffici_INSTANCE_6TA6OwuprQd0/dettaglioufficio/TA2
Following link: https://www.agenziaentrate.gov.it/portale/web/guest/uffici/-/uffici/uffici_INSTANCE_6TA6OwuprQd0/dettagliouffT/TA3C/cr/915/iur/TA3C
 
Following link: https://www.agenziaentrate.gov.it/portale/web/guest/uffici/-/uffici/uffici_INSTANCE_6TA6OwuprQd0/dettagliouffT/TA3L/cr/915/iur/TA3L
 
Following link: https://www.agenziaentrate.gov.it/portale/web/guest/uffici/-/uffici/uffici_INSTANCE_6TA6OwuprQd0/dettagliouffT/TAE/cr/915/iur/TAE
Following link: https://www.agenziaentrate.gov.it/portale/web/guest/uffici/-/uffici/uffici_INSTANCE_6TA6OwuprQd0/dettagliouffT/TAF/cr/915/iur/TAF
Following link: https://www.agenziaentrate.gov.it/portale/web/guest/uffici/-/uffici/uffici_INSTANCE_6TA6OwuprQd0/dettagliouffT/TAJ/cr/915/iur/TAJ
Following link: https://www.agenziaentrate.gov.it/portale/web/guest/uffici/-/uffici/uffici_INSTANCE_6TA6OwuprQd0/dettagli

In [23]:
url = 'https://www.agenziaentrate.gov.it/portale/web/guest/dr-valle-daosta'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0"}

resp = httpx.get(url, headers=headers)

if resp.status_code == 200:
    html = HTMLParser(resp.text)
    region_name = "Valle d'Aosta"   
    province = html.css("a.btn.btn-primary.btn-entrypoint")
    for prov in province:
        print(prov.text())
        link_prov = prov.attributes['href']
        full_link = httpx.URL(url).join(link_prov)
        print(f"Following link: {full_link}")

        resp_prov = httpx.get(full_link, headers=headers)
        if resp_prov.status_code == 200:

            # da modificare per prendere tutti gli uffici disponibili
            html_menu = HTMLParser(resp_prov.text)
            menu = html_menu.css("li.submenu")
            
            for direnzione in menu[2:4]:
                link_direzioni = direnzione.css('a')
                link_dir = link_direzioni[0].attributes['href']
                print(f"Following link: {link_dir}")
                resp_dir = httpx.get(link_dir, headers=headers)

                html_content = resp_dir.text
                soup = BeautifulSoup(html_content, 'html.parser')

                uffici = soup.select("h1.underline")
                ufficio = uffici[0].text

                if 'Ufficio Controlli' in ufficio or 'Ufficio Legale' in ufficio:
                    print(' ')

                else:    

                    # Locate the specific div containing "Informazioni utili"
                    divs = soup.select('div.panel.panel-default.panel-collapsible')
                    

                    for div in divs:
                        informazioni_div=''
                        title_element = div.select_one('a.panel-title')
                        if title_element and 'Informazioni utili' in title_element.text:
                            informazioni_div = div
                            break

                    try:
                        if informazioni_div:

                            paragraphs = informazioni_div.find_all('p')

                        for paragraph in paragraphs:
                            info.append({
                                'region': region_name,
                                'province': ufficio,
                                'info': paragraph.get_text()
                            })
                            paragraph=''
                        informazioni_div='' 
                        paragraphs=''   

                    except Exception as e:
                        print(f"Error finding 'Collapse-2' <p> element: {e}")
                        errors.append({
                                'region': region_name,
                                'province': prov.text()})
        else:
            print(f"Failed to fetch province page: {full_link}")
else:
    print("Failed to fetch main page.")

Aosta
Following link: https://www.agenziaentrate.gov.it/portale/web/guest/uffici19/-/uffici/uffici_INSTANCE_nC8vEKuwqWiP/dettaglioufficio/T4D
Following link: https://www.agenziaentrate.gov.it/portale/web/guest/uffici19/-/uffici/uffici_INSTANCE_nC8vEKuwqWiP/dettaglioufficio/T4D
Following link: https://www.agenziaentrate.gov.it/portale/web/guest/uffici19/-/uffici/uffici_INSTANCE_nC8vEKuwqWiP/dettaglioufficio/T4E


In [26]:
filtered_data = []

# Step 1: Filter entries that contain 'Competenze' or 'Competenza' in 'info'
for entry in info:
    if 'Competenze' in entry['info'] or 'Competenza' in entry['info']:
        filtered_data.append({'region': entry['region'], 'province': entry['province'], 'info': entry['info']})

# Step 2: Ensure only one entry per unique combination of region and province, preferring 'Competenza'
final_data = {}
for entry in filtered_data:
    key = (entry['region'], entry['province'])
    if key not in final_data:
        final_data[key] = entry
    else:
        # If the current entry contains 'Competenza', replace the existing entry
        if 'Competenza' in entry['info']:
            final_data[key] = entry
            

# Convert the final_data dictionary back to a list
result = list(final_data.values())

result


[{'region': 'Abruzzo',
  'province': "Direzione Provinciale di L'AQUILA - Ufficio Territoriale AVEZZANO",
  'info': "Competenza territoriale: Aielli, Avezzano, Balsorano, Bisegna, Canistro, Capistrello, Cappadocia, Carsoli, Castellafiume, Celano, Cerchio, Civita d'Antino, Civitella Roveto, Cocullo, Collarmele, Collelongo, Gioia dei Marsi, Lecce Nei Marsi, Luco dei Marsi, Magliano de' Marsi, Massa d'Albe, Morino, Opi, Oricola, Ortona dei Marsi, Ortucchio, Ovindoli, Pereto, Pescasseroli, Pescina, Rocca di Botte, San Benedetto dei Marsi, San Vincenzo Valle Roveto, Sante Marie, Scurcola Marsicana, Tagliacozzo, Trasacco, Villavallelonga"},
 {'region': 'Abruzzo',
  'province': "Direzione Provinciale di L'AQUILA - Ufficio Territoriale CASTEL DI SANGRO",
  'info': 'Competenza territoriale: Alfedena, Ateleta, Barrea, Castel di Sangro, Civitella Alfedena, Pescocostanzo, Rivisondoli, Roccaraso, Scontrone, Villetta Barrea'},
 {'region': 'Abruzzo',
  'province': "Direzione Provinciale di L'AQUILA -

In [27]:
sezioni = pd.DataFrame(result)
sezioni.to_excel('../Resources/sezioni_territoriali_2.xlsx', index=False)

In [ ]:
sezioni

In [ ]:
"""info = pd.read_excel('Resources/sezioni_territoriali.xlsx')

def remove_text(info_column):
    text_to_remove = [".",
        ", i servizi catastali e di pubblicità immobiliare, i servizi tecnico-estimativi e l'Osservatorio del Mercato Immobiliare.",
        "e' strutturata in un ufficio controlli, un Ufficio Legale",
        "competenze: la direzione provinciale cura l'attività di informazione e assistenza ai contribuenti, la gestione dei tributi, l'accertamento, la riscossione e la trattazione del contenzioso",
        ", i servizi catastali e di pubblicità immobiliare, i servizi tecnico-estimativi e l'osservatorio del mercato immobiliare",
        ", i servizi catastali e di pubblicità immobiliare, i servizi tecnico-estimativi e l’osservatorio del mercato immobiliare"
    ]
    for text in text_to_remove:
        info_column = info_column.str.replace(text.lower(), '', regex=False).str.strip()
    return info_column

info = info.applymap(lambda x: x.lower() if isinstance(x, str) else x)

# Apply the function to the desired column, assuming the column name is 'info'
info['info'] = remove_text(info['info'])


# Set pandas display options to show all rows and columns
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

# Verify the changes
info"""

In [ ]:
"""info = pd.read_excel('Resources/sezioni_territoriali.xlsx')
def remove_text(info):
    text_to_remove = [
        "Competenze: La Direzione provinciale cura l'attività di informazione e assistenza ai contribuenti, la gestione dei tributi, l'accertamento, la riscossione e la trattazione del contenzioso",
        ", i servizi catastali e di pubblicità immobiliare, i servizi tecnico-estimativi e l'Osservatorio del Mercato Immobiliare.",
        ".E' strutturata in un Ufficio Controlli, un Ufficio Legale"
    ]
    for text in text_to_remove:
        info = info.replace(text, '')
    return info

# Remove specified text from each dictionary in the list
for item in info:
    item['info'] = remove_text(item['info'])

info"""